## Object Oriented Design <p> December 7, 2018

In this lab we will explore **Object Oriented Programming**. Object oriented programming is a method of programming that attempts to model some process or thing in the world as a class or object. Conceptually, you can think of a class or object as something that has data and can perform operations on that data. With object oriented programming, the goal is to encapsulate your code into logical groupings using classes so that you can reason about your code at a higher level. Before we get ahead of ourselves, though, let's see an example:

In [ ]:
class Pet:
    def __init__(self, name, animal_type, age):
        self.name = name
        self.animal_type = animal_type
        self.age = age
        self.tricks = []
        
    def add_trick(self, trick):
        self.tricks.append(trick)

We've made a *class* called Pet. A pet can have a number of instance variables: name, animal_type, age, and tricks. Copy the code and try each of the following commands:

In [ ]:
myFirstPet = Pet("Fido","Dog","6")
myFirstPet.name
myFirstPet.add_trick("roll over")
myFirstPet.tricks
mySecondPet = Pet("Oscar","Cat","4")
mySecondPet.add_trick("napping")
mySecondPet.tricks

In [ ]:
class Pet2:
    tricks = []

    def __init__(self, name):
        self.name = name

    def add_trick(self, trick):
        self.tricks.append(trick)

### Q1: What is the difference between Pet2 and Pet? 

Run the following commands:

In [ ]:
myFirstPet = Pet2("Fido")
myFirstPet.name
myFirstPet.add_trick("roll over")
myFirstPet.tricks
mySecondPet = Pet2("Oscar")
mySecondPet.add_trick("napping")
mySecondPet.tricks

### Q2: Describe what you observe in the output. Is it what you expected? Is the behavior desirable?

Let's construct a new class which **inherits** from the Pet class. The new Dog class contains every attribute from the Pet class, plus a new attribute called chases_cats. It also has a new method. 

In [56]:
class Dog(Pet):
    def __init__(self, name, age, chases_cats):
        self.chases_cats = chases_cats
        Pet.__init__(self, name,"dog",age)
        
    def chases_cats(self):
        return self.chases_cats

Test the following commands:

In [ ]:
dog1 = Pet("Rowdy","Dog",11)
dog2 = Dog("Isabelle", 3, True)
isinstance(dog1,Pet)
isinstance(dog1,Dog)
isinstance(dog2,Pet)
isinstance(dog2,Dog)

### Q3: Make another new class which inherits from Pet. Make a new class which inherits from Dog. Include a new instance variable or method (or both) for each new class.

Now that we've seen a small example, let's think about a more difficult problem. Suppose we want to simulate a dice game. The game will work in the following way:
    * A player initially rolls a random hand of dice
    * The player gets two chances to improve the hand by rerolling some or all of the dice
    * At the end of the hand, the values in the hand are scored according to the following values. Two pairs earns 5 points. A three of a kind earns 8 points. A full house earns 12 points. A straight (1-5 or 2-6) earns 20 points. And a five of a kind earns 30 points.
    
When designing this program in an object-oriented fashion we need to think about candidate **objects**. One object could be a set of dice. 

In [ ]:
import random

class Dice:
    def __init__(self):
        self.dice = [0]*5
        self.rollAll()
        
    def roll(self,which):
        for pos in which:
            self.dice[pos] = random.randrange(1,7)
    
    def rollAll(self):
        self.roll(range(5))
        
    def values(self):
        return self.dice[:]
    

In English: This code creates a class called Dice. Each set of dice is initialized to be 5 zeros, and then the rollAll method is called to obtain 5 random rolls of the dice. There are two other methods: roll, which only rolls a subset of the dice, and values, which displays the 5 dice values. Copy the code and run the following commands:

In [ ]:
myDice = Dice()
myDice.values()
myDice.rollAll()
myDice.values()
myDice.roll([4])
myDice.values()

We also want to be able to determine the *score* of a hand of dice. This method relies on counting the number of duplicates in the set of 5 dice. 

### Q4: In the code below, what is the value of *counts* for the following cases? v = [1,2,2,3,5], v = [6,4,3,6,5], v = [1,1,1,1,1].

In [ ]:
v = [1,2,2,3,5]

counts = [0] * 7
for value in v:
    counts[value] = counts[value] + 1

counts

Now we can use counts in our program. Add the following method to the Dice class.

In [36]:
    def score(self):
        counts = [0] * 7
        for value in self.dice:
            counts[value] = counts[value] + 1
            
        if 5 in counts:
            return "Five of a Kind", 30
        elif 4 in counts:
            return "Four of a Kind", 15
        elif (3 in counts) and (2 in counts):
            return "Full House", 12
        elif 3 in counts:
            return "Three of a Kind", 8
        elif not (2 in counts) and (counts[1] ==0 or counts[6] == 0):
            return "Straight",20
        elif counts.count(2) == 2:
            return "Two Pairs", 5
        else:
            return "Nothing", 0

Expirement with the code at this point. Make sure that when you roll a hand it scores properly. Notice that in the current set up, you can re-roll the hand as many times as you want, which means that there is an important component missing: The game itself.

The game will need a set of dice, as well as some way to manage the rolling. The TextInterface class allows us to play the game using the terminal or console. 

In [ ]:
class DiceGame:
    def __init__(self,interface):
        self.dice = Dice()
        self.interface = interface
        
    def run(self):
        while self.interface.wantToPlay():
            self.playRound()
        self.interface.close()
    
    def playRound(self):
        self.doRolls()
        result,score = self.dice.score()
        self.interface.showResult(result,score)
        
    def doRolls(self):
        self.dice.rollAll()
        roll = 1
        self.interface.setDice(self.dice.values())
        toRoll = self.interface.chooseDice()
        while roll < 3 and toRoll != []:
            self.dice.roll(toRoll)
            roll = roll + 1
            self.interface.setDice(self.dice.values())
            if roll < 3:
                toRoll = self.interface.chooseDice()
                
class TextInterface:
    def __init__(self):
        print("Welcome to the Dice Game")
        
    def setDice(self,values):
        print("Dice:",values)
        
    def wantToPlay(self):
        ans = input("Do you want to play? ")
        return ans[0] in "yY"
    
    def close(self):
        print("\nThanks for playing!")
    
    def showResult(self,msg,score):
        print("{0}. You win {1} points.".format(msg,score))
        
    def chooseDice(self):
        return eval(input("Enter list of which to change ([] to stop)"))

The three classes: Dice, DiceGame, and TextInterface are what we need to play our game. Save these codes to a file named DiceApp.py. Then run the following code in python:

In [ ]:
from DiceApp import DiceGame
from DiceApp import TextInterface

inter = TextInterface()
app = DiceGame(inter)
app.run()

### Q5: Imagine that we modify our game in the following way. Suppose that a player starts the game with 100 points. Each time you play, you lose 10 points. (Though you may gain back points each time you play through the same scoring system). The player can no longer play if they have less than 10 points. 

### In what ways would the three classes need to change? Implement those changes. To start you off, the initialization of the DiceGame class needs to track the total points:

In [ ]:
def __init__(self,interface):
        self.dice = Dice()
        self.interface = interface
        self.points = 100